<a href="https://colab.research.google.com/github/cybermin/pythonbasic/blob/master/work/06/ex22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

해결문제)

daum 영화 사이트에서 현재 상영작의 목록을 추출하고 상영영화를 선택하면 해당 영화의 상위 10개의 리뷰 내용을 감성분석하시오.
+ https://movie.daum.net/premovie/released





In [1]:
#한글 폰트 사용 : colab , 런타임 다시 시작 하기
%config InlineBackend.figure_format = 'retina'

!apt -qq -y install fonts-nanum

The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [55]:
from urllib.request import urlopen
from urllib import parse

from bs4 import BeautifulSoup as bs

import matplotlib.pyplot as plt

import matplotlib as mpl
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

import re

In [3]:
#문자열에서 charset요소값을 찾아줌
def charEncode(st) :
    pt = r'charset=[\"\']*([\w-]+)'

    charcode  = re.search(pt, st)
    if charcode : 
        cc = charcode.group(1)
    else :
        cc = 'utf-8'

    return cc

#해당 페이지를 decode한 html문서 가져오기
def urlHtml(url):
    data = urlopen(url).read()
    urlcc = charEncode(str(data))
    html = data.decode(urlcc, errors="replace")

    html = bs(html, 'html.parser')
    return html

In [36]:
def review(url) :
    rt = []
    #/moviedb/main?movieId=134684
    # => https://movie.daum.net/moviedb/grade?movieId=134684

    url = url.replace('main', 'grade')
    url = 'https://movie.daum.net' + url
    reviewhtml = urlHtml(url).select('.list_review > li')
    for item in reviewhtml :
        desc_review = item.select_one('.desc_review').text
        desc_review = desc_review.strip().replace('\n','').replace('\r','')
        rt.append(desc_review)

    return rt


In [38]:
review('/moviedb/main?movieId=134684')

['',
 '코로나 시기에 일부러 시간내서 돈 내서 보러 갔는데 무슨 tv드라마보다 재미가 없노.',
 '평점이 5점이나 되나요? 너무 높아서 원래 2점짜리인데, 0점 줍니다.',
 '재미없음',
 '내용은 없으나 연기력은 짱',
 '우리 살아있다',
 '볼 영화가 없어서 예매율 1위인 영화.예고편이 전부인 영화임.좀비가 한국에서 뜨니까 만들어진 아류작이며 보는 내내 번히 예성되는 내용, 뻔한  전개 때문에 극장 빈자리를 둘러보면서 하품하기는 처음.고등학교, 대학교 영화동아리에서 으싸 으싸햐서 여름 방학동안 만든 영화 같음.영화 음악도 어디서 둘운거 같은데??? 생각이 듬',
 '',
 '',
 '내용은 평범한데 유아인 연기력의 점수준거임']

In [40]:
url = 'https://movie.daum.net/premovie/released'

html = urlHtml(url)


In [45]:
ul = html.select('.list_boxthumb > li')
mvdt = {}
for li in ul :
    lt = []
    #영화정보 : 태그로 찾기
    ainfo = str(li.select_one('a'))

    #영화정보 추출
    href = re.search(r'href=\"([\w\/?=]+)', ainfo).group(1)

    #영화명 : 태그로 찾기
    mvname = li.select_one('strong').text.replace(',', '')
    lt = review(href)

    mvdt[mvname] = lt

In [47]:
with open('/content/drive/My Drive/PythonWork/PythonBasic/work/06/mv2.csv','w') as fp :
    for key , value in mvdt.items() :
        for item in value :
            s = key + "," + item + "\n"
            fp.write(s)

### 감성분석

In [53]:
import json
def adamsAi(myquery, mytype) :
    mykey = ''

    url = 'http://api.adams.ai/datamixiApi/omAnalysis?key=' + mykey 
    url = url +'&query=' + parse.quote(myquery )
    url = url +'&type=' + mytype    
  
    data = urlopen(url).read().decode('utf-8')
    dt = json.loads(data)

    if mytype == '0' :
        score = dt["return_object"]["score"]
        label = dt["return_object"]["label"]
    else :
        score = dt["return_object"]["Result"][0][0] 
        label = dt["return_object"]["Result"][0][1] 
        
    return score, label

In [65]:
with open('/content/drive/My Drive/PythonWork/PythonBasic/work/06/mv2.csv','r', encoding='utf-8') as fp :
    data = fp.readlines()

data2 = []
for item in data :
    item = item.replace('\n','')
    items = item.split(',')
    
    if len(items[1].strip()) > 0 :
        score, label = adamsAi(items[1][:100], '0') 
        #score, label = 0, '1'
        item = items[0]+ ',' + items[1] + ',' + str(score) + ','+ label + '\n'
    else : item = items[0]+ ',' + items[1] + '\n'
    
    data2.append(item)

In [ ]:
data2

In [67]:
with open('/content/drive/My Drive/PythonWork/PythonBasic/work/06/mv3.csv','w') as fp :
    for item in data2 :
        fp.write(item)